In [1]:
from vkapi.methods import *
from vkapi import VKError

# Additional libs 
from random import randint
from time import sleep
from collections import Counter

## Public methods
These methods do not require access token. Therefore they could be used without authentication

## VK Top 
Here we will try to find the top of vk: ie. the users and groups with the largest number of followers.
### Step 1.
Collect subscriptions (who user follows) of random $N$ users. Just a dictionary `{'an_id': n_times_appeared}`

In [2]:
help(users.getSubscriptions)

Help on function getSubscriptions in module vkapi.methods.users:

getSubscriptions(user_id=None, extended=None, offset=None, count=None, fields=None)
    Returns a list of IDs of users and communities followed
    by the user.
    https://vk.com/dev/users.getSubscriptions



In [5]:
def get_subscriptions(user_id):
    uids = []
    gids = []
    offset = 0
    while True:
        users_and_groups = users.getSubscriptions(user_id, offset=offset, count=1000)
        if len(users_and_groups) == 0:
            break
        uids_ = [item['uid'] for item in users_and_groups if 'uid' in item]
        gids_ = [item['gid'] for item in users_and_groups if 'gid' in item]
        uids.extend(uids_)
        gids.extend(gids_)
        offset += 1000
        sleep(0.2) # No more than 3 requests per second.
    return uids, gids

In [6]:
N = 1000 # number of users to collect
user_count = 0
uids = Counter()
gids = Counter()

while user_count < N:
    try:
        user_id = randint(0, 2*10**8)
        current_uids, current_gids = get_subscriptions(user_id)
        uids.update(current_uids)
        gids.update(current_gids)
        user_count += 1
    except VKError:
        # Invalid user id or related
        pass
    
    print('\r{}/{}'.format(user_count, N), end='')

1000/1000

In [14]:
print("Top Users")
for u, _ in uids.most_common()[:5]:
    print("\thttps://vk.com/id{}".format(u))
    
print("Top Groups")
for g, _ in gids.most_common()[:5]:
    print("\thttps://vk.com/club{}".format(g))

Top Users
	https://vk.com/id1
	https://vk.com/id45269508
	https://vk.com/id53083705
	https://vk.com/id20020126
	https://vk.com/id43787823
Top Groups
	https://vk.com/club22798006
	https://vk.com/club27895931
	https://vk.com/club43215063
	https://vk.com/club12382740
	https://vk.com/club460389


### Results
The above descripbed methods gives the approximate short list of top users and communities. 

The above described approach uses the simpliest and the slowliest methods. If you want to accelerate the data collection use *__execute__* method. It executes the *__ES__* like script. It allows to accelerate the API calls up to 30 times.